<div><img style="float: right; width: 120px; vertical-align:middle" src="https://www.upm.es/sfs/Rectorado/Gabinete%20del%20Rector/Logos/EU_Informatica/ETSI%20SIST_INFORM_COLOR.png" alt="ETSISI logo" />


# Word embedding con Word2Vec<a id="top"></a>

<i><small>Autor: Alberto Díaz Álvarez<br>Última actualización: 2023-03-14</small></i></div>
                                                  

***

## Introducción

Comenzamos con los más importante de todo: _Word embedding techniques_ es una forma de decir _representar palabras de forma numérica_ pero con más gancho. Y una vez dicho esto, vamos a programar un proceso de aprendizaje de _embeddings_ a partir de corpus de texto. Nos centraremos en una técnica denominada _Word2Vec_, aunque ya hemos visto que hay más.

_Word2vec_ se basa en una red neuronal que genera la matriz usando un entrenamiento supervisado sobre un problema de clasificación. El artículo donde se presenta el método es [Efficient Estimation of Word Representations in Vector Space (Mikolov et al.,2013)](https://arxiv.org/pdf/1301.3781.pdf) y es un método que se usa con bastante éxito a la hora de medir **similitud sintáctica y semántica de palabras**.

El artículo explora dos modelos diferentes: _Bag-of-Words_ y _Skip-gram_. El más usado es este último, y será el que veamos en este ejercicio.

La idea del _Skip-gram_ es la siguiente: dada una palabra (que denominaremos _palabra de contexto_), queremos entrenar un modelo de tal manera que sea capaz de predecir una palabra perteneciente a una ventana de tamaño $N$. Por ejemplo, suponiendo una ventana de tamaño $N = 3$ y dada la siguiente frase:

> Todos <span style="color:red">esos momentos se</span> **perderán** <span style="color:red">en el tiempo</span> como lágrimas en la lluvia_

La _palabra de contexto_ sería **perderán**, y entrenaríamos al modelo para predecir una de las palabras existentes dentro de la ventana especificada, es decir, una de entre `['esos', 'momentos', 'se', 'en', 'el', 'tiempo']`.

## Objetivos

En este _notebook_ crearemos un _embedding_ a partir de la técnica _skip-gram_ de _Word2Vec_.

## Imports y configuración

A continuación importaremos las bibliotecas que usaremos a lo largo del _notebook_:

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

import matplotlib.pyplot as plt

Asímismo, configuramos algunos parámetros para adecuar la presentación gráfica:

In [2]:
plt.style.use('ggplot')
plt.rcParams.update({'figure.figsize': (16, 9),'figure.dpi': 100})

***

## Construcción del corpus

Hemos descargado un [corpus de textos](https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/data) sobre comentarios tóxicos (que usaremos más adelante en otros notebooks). El primer paso será leerlo y separarlo en frases. Este paso generalmente requiere bastante tiempo porque las fuentes originales de datos no vienen completamente limpias.

Veamos primero el corpus del fichero con el conjunto de entrenamiento

In [3]:
df = pd.read_csv('Datasets/Toxic/train.csv')
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


Para nuestro cometido (crear un _embedding_), nos da igual el output del modelo o el id de los ejemplos; Nosotros queremos los textos, así que vamos a extraerlos, eliminando los blancos que puedan tener al principio y al final.

In [4]:
corpus = df['comment_text'].str.strip()
corpus.head()

0    Explanation\nWhy the edits made under my usern...
1    D'aww! He matches this background colour I'm s...
2    Hey man, I'm really not trying to edit war. It...
3    "\nMore\nI can't make any real suggestions on ...
4    You, sir, are my hero. Any chance you remember...
Name: comment_text, dtype: object

La variable `corpus` apunta a una serie con todas las frases de nuestro conjunto. Vamos a tokenizar cada uno de los comentarios, convirtiéndolos en una lista de palabras. Para ello usaremos la función `tokenizer` incluida en keras, aunque es importante entender que este paso no es trivial y seguramente requiera de mucho preproceso para tener un conjunto de datos de calidad (e.g. lematización, $n$-gramas, ...).

In [5]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(corpus)

En este momento, nuestro _tokenizer_ ha procesado todos los comentarios y ha extraído todas las palabras, asignándoles un identifficador a cada una. Las almacenaremos en dos diccionarios para poder convertirlas en enteros (para identificar la palabra) y en palabras (una vez tengamos el entero

In [6]:
word_index = tokenizer.word_index
index_word = {index: word for word, index in word_index.items()}

print(f'word2id: {dict(list(word_index.items())[0:4])} ...')
print(f'id2word: {dict(list(index_word.items())[0:4])} ...')

word2id: {'the': 1, 'to': 2, 'of': 3, 'and': 4} ...
id2word: {1: 'the', 2: 'to', 3: 'of', 4: 'and'} ...


Por último, cada uno de los comentarios del corpus será transformado en una lista de enteros donde cada token del mismo se reemplazará por el entero que representa. Obtendremos también el tamaño de nuestro vocabulario a partir del número de palabras identificadas.

Para convertir una cadena de texto en una secuencia de palabras podemos usar la función de Keras `tf.keras.preprocessing.text.text_to_word_sequence(text)`. A partir de ahí sacar el índice de cada palabra es trivial.

In [7]:
sentences = [
    [word_index[w] for w in tf.keras.preprocessing.text.text_to_word_sequence(text)]
    for text in corpus
]
vocab_size = len(word_index) + 1

print(f'Corpus sentences: {len(sentences)} sentences')
print(f'Vocabulary Size: {vocab_size} words')
print(f'Sentence example: {sentences[9]}')

Corpus sentences: 159571 sentences
Vocabulary Size: 210338 words
Sentence example: [10960, 15, 13, 242, 4, 53, 19, 1815, 2, 142, 3, 65704]


## Generador de skip-grams

Ahora generaremos los _skip-grams_. La idea es, de todas las frases del corpus (cada `sentence` de `sentences`) y dada una ventana de acción, sacar su contexto (las palabras alrededor) para determinar para cada par de palabras si son o no contextuales.

In [8]:
WINDOW_SIZE = 5

dataset = None
for sentence in sentences[:5000]:
    s = tf.keras.preprocessing.sequence.skipgrams(
        sentence,
        vocabulary_size=vocab_size,
        window_size=WINDOW_SIZE,
    )

    if s[0] and s[1]:
        X = np.array(s[0])
        Y = np.array(s[1]).reshape(-1, 1)
        subset = np.hstack((X, Y))

        dataset = subset if dataset is None else np.vstack((dataset, subset))

print(f'Dataset shape: {dataset.shape}')
print(dataset)

Dataset shape: (6610128, 3)
[[    31    365      1]
 [  2756 158196      0]
 [    51   4511      1]
 ...
 [   102   1391      1]
 [    97 176286      0]
 [     9    217      1]]


Se puede ver que se ha generado, para cada par de palabras, si son (1) o no (0) contextuales. Esto es porque la función `skipgrams` transforma una secuencia de palabras (en realidad de enteros) en tuplas de la forma:

- (palabra, palabra en el contexto), label 1 (positivo, son contextuales).
- (palabra, palabra aleatoria del vocabulario), label 0 (no son contextuales).

## Creación y entrenamiento del modelo

Ya tenemos un dataset con inputs y sus respectivos outputs. Ahora el objetivo es entrenar un modelo que es capaz de determinar si dos palabras pertencen al mismo contexto.

Para ello crearemos una capa de embedding que transformara las palabras en su vector de características. Las palabras serán aquellas que son o no contextuales, y se determinará lo cerca (más contextuales) o lejos (menos contextuales) que están, usando una medida de distancia (producto escalar).

Por último, la salida de la red será una única neurona que se activará o no si son contextuales.

Esta arquitectura forzará a que las palabras más contextuales estén más cerca, y por tanto que sus vectores de características sean más similares. Se espera así que la matriz de pesos de la capa de embedding converja a una representación de las características de las palabras.

In [9]:
EMBEDDING_DIM = 5

input_target = tf.keras.layers.Input((1,))
input_context = tf.keras.layers.Input((1,))

embedding_layer = tf.keras.layers.Embedding(
    input_dim=vocab_size,
    output_dim=EMBEDDING_DIM,
    input_length=1,
    name='embedding',
)

target_embedding = embedding_layer(input_target)
target_embedding = tf.keras.layers.Reshape((EMBEDDING_DIM, 1))(target_embedding)
target_embedding = tf.keras.layers.Dropout(0.5)(target_embedding)

context_embedding = embedding_layer(input_context)
context_embedding = tf.keras.layers.Reshape((EMBEDDING_DIM, 1))(context_embedding)
context_embedding = tf.keras.layers.Dropout(0.5)(context_embedding)

hidden_layer = tf.keras.layers.Dot(axes=1, normalize=True)([target_embedding, context_embedding])
hidden_layer = tf.keras.layers.Reshape((1,))(hidden_layer)

output = tf.keras.layers.Dense(1, activation='sigmoid')(hidden_layer)

model = tf.keras.Model(inputs=[input_target, input_context], outputs=output)
model.summary()
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(clipnorm=0.0001))

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 embedding (Embedding)          (None, 1, 5)         1051690     ['input_1[0][0]',                
                                                                  'input_2[0][0]']                
                                                                                                  
 reshape (Reshape)              (None, 5, 1)         0           ['embedding[0][0]']          

Ya sólo nos queda entrenar el modelo. Para ello, lo entrenaremos con cada uno de los _skip-grams_ generados anteriormente. Usaremos una separación de validación del 10% y entrenaremos durante 10 epochs.

**Este paso es muy costoso**, y se puede tirar bastantes minutos (horas) así que, o tenemos una máquina potente, o mejor lo dejamos aquí.

In [ ]:
history = model.fit([dataset[:, 0], dataset[:, 1]], dataset[:, 2], epochs=50, validation_split=0.1)

Veamos la evolución del entrenamiento:

In [ ]:
plt.plot(history.history['loss'], label='Training')
plt.plot(history.history['val_loss'], label='Validation')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()

## Embeddings

Una vez entrenado el modelo, ya tenemos una matriz con los pesos de las características para cada palabra. Para ver una representación podemos cogerlos directamente e imprimirlos en un dataframe.

In [ ]:
weights = embedding_layer.get_weights()[0][1:]

df = pd.DataFrame(weights, index=index_word.values())
df.head(10).style.background_gradient(cmap ='hot').format('{:.2f}')

Vamos a hacer una búsqueda con las palabras más parecidas a una dada usando, por ejemplo, la distancia euclídea de sus vectores

In [ ]:
NUM_CLOSEST_WORDS = 10
WORD = 'man'

v1 = weights[word_index[WORD] - 1]
words = sorted(
    [word for word in word_index.keys()],
    key=lambda w: np.linalg.norm(v1 - weights[word_index[w]-1])
)
df.loc[words[:NUM_CLOSEST_WORDS + 1], :].style.background_gradient(cmap ='hot').format('{:.2f}')

## Conclusiones

En conclusión, hemos implementado un embedding utilizando la técnica _skip-grams_ de _word2vec_ y hemos demostrado su eficacia para representar palabras de manera más significativa en un espacio vectorial. Esta técnica es capaz de capturar la semántica de las palabras, representándolas en un espacio vectorial de dimensión menor a la que ocuparía con una representación _one-hot_.

Ojo, también hemos comprobado que **es una técnica muy costosa**, y que por tanto no tiene mucho sentido (en general) ya que disponemos de muchos _embeddings_ ya preparados para descargar.

***

<div><img style="float: right; width: 120px; vertical-align:top" src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by-nc-sa.png" alt="Creative Commons by-nc-sa logo" />

[Volver al inicio](#top)

</div>